In [49]:
%pip install sympy
import numpy as np
from IPython.display import display, Markdown, Latex
from sympy import *

# determinação dos parâmetros do circuito
R, L, C = 200, 50e-3, 0.2e-6

# determinacao da tensão inicial do capacitor
v_0 = 12

# determinação da corrente inicial do indutor
iL_0 = 30e-3

# determinação da corrente inicial do resistor
iR_0 = v_0 / R

# determinação da corrente de deslocamento do capacitor
# no instante t = 0^+
iC_0 = -iL_0 - iR_0

# determinação da derivada primeira da tensão no instante de tempo t = 0^+
dv_0dt = iC_0 / C

# definição das funções para cálculo de alpha, omega e consequentemente
# das raízes características s_1 e s_2 do circuito em função dos parametros
# RLC do sistema
alpha = lambda R, C: np.round(1/(2*R*C))
omega_0 = lambda L, C: np.round(1/np.sqrt(L*C))
s_1 = lambda R, L, C: np.round(
    -alpha(R,C) + np.sqrt(
        np.power(alpha(R,C), 2) - np.power(omega_0(L,C), 2) + 0j
    )
)
s_2 = lambda R, L, C: np.round(
    -alpha(R,C) - np.sqrt(
        np.power(alpha(R,C), 2) - np.power(omega_0(L,C), 2) + 0j
    )
)

# cálculo das raízes características s_1, s_2 em função de RLC
S_1, S_2 = np.array([s_1(R, L, C), s_2(R, L, C)])

'''
    construção do sistema linear
        v0 = A_1 + A_2
        v\'0 = S_1 * A_1 + S_2 * A_2
    para obtenção das constantes A_1 e A_2
'''
coeficientes = np.array([
    [1,     1],
    [S_1, S_2]
])
termos_independentes = np.array([
    [v_0],
    [dv_0dt]
])

# obtenção das constantes A_1 e A_2 pela resolução do sistema linear
# no numpy
A_1, A_2 = np.linalg.solve(coeficientes, termos_independentes).ravel()

# definicao de uma variavel simbolica t, necessária para obter
# a funcao v(t) e posteriormente, a função das correntes
# do indutor i_L(t) e do resistor i_R(t)
t = Symbol('t')

# definicao da funcao v(t)
v = A_1 * exp(S_1 * t) + A_2 * exp(S_2 * t)

# definicao da corrente i_R(t)
iRT = v / 200

# definicao da corrente i_L(t)
iLT = (1/L) * (integrate(v) - integrate(v).subs(t, 0)) + iL_0
# arredondando os pontos flutuantes em i_L(t)
for term in preorder_traversal(iLT):
    if isinstance(term, Float):
        iLT = iLT.subs(term, round(term, 3))

# definicao da corrente i_L(t) pela LKC
# pela LKC, iLTLKC = -iRT - iCT
# já conhecemos iRT. iCT, a corrente do capacitor no tempo t
# é C dv/dt. Temos vt
dvdt = diff(v)
iCT = C * dvdt
iLTLKC = -iRT - iCT
# arredondando os pontos flutuantes em i_L(t) pela LKC
for term in preorder_traversal(iLTLKC):
    if isinstance(term, Float):
        iLTLKC = iLTLKC.subs(term, round(term, 3))

md = '''

![](https://i.imgur.com/23zF3dF.png)

**Solução**

Sabemos que a expressão para v(t) é:

\\\\[
\\begin{{align}}
    v(t) &= {vt} V, t \geq 0 \\therefore \\\\
    v(t) &= ({{{A_1}}} e^{{{S_1} t}} + {{{A_2}}} e^{{{S_2} t}}) V, t \geq 0
\\end{{align}}
\\\\]

onde \\\\(exp(\\chi)\\\\) representa a função exponencial natural \\\\(e^{{\\chi}}\\\\)

Sabemos também que 

\\\\[
\\begin{{align}}
    i_{{R}}(t) &= \\frac{{v(t)}}{{R}} = \\frac{{{vt}}}{{200}} \\therefore \\\\
    i_{{R}}(t) &= {iRT} \\therefore \\\\
    i_{{R}}(t) &= (-70 e^{{-5000t}} + 130 e^{{-20000 t}})mA, t \\geq 0
\\end{{align}}
\\\\]

Agora devemos encontrar a corrente do indutor \\\\(I_{{L}}\\\\).

Podemos encontrá-la de duas formas:

1. Pela definição da corrente no indutor que preconiza: \\\\(i_{{L}}(t) = \\frac{{1}}{{L}} \\int_{{0}}^{{t}}{{v(\\chi) d\\chi}} + I_{{0}} \\\\)

2. Pela determinação da corrente no ramo capacitivo e, por fim, pela lei de kirchoff das correntes que estabelece: \\\\(i_{{L}}(t) = - i_{{R}}(t) - i_{{C}}(t)\\\\)

Vamos fazer a primeira forma inicialmente para validá-la com a segunda.

A primeira forma é definida nesta célula jupyter pela variável iLT:

\\\\[
    i_{{L}}(t) = {iLT}
\\\\]

A segunda forma, definida por:

\\\\[
\\begin{{align}}
    i_{{L}}(t) &= - i_{{R}}(t) - i_{{C}}(t) \\therefore \\\\
    i_{{L}}(t) &= - {iRT} - {iCT} \\therefore \\\\
    i_{{L}}(t) &= {iLTLKC}
\\end{{align}}
\\\\]

Verifica a primeira.
'''.format(
    A_1 = int(A_1.real),
    A_2 = int(A_2.real),
    S_1 = int(S_1.real),
    S_2 = int(S_2.real),
    vt = v,
    iRT = iRT,
    iLT = iLT,
    iCT = iCT,
    iLTLKC = iLTLKC
)
display(Markdown(md))



![](https://i.imgur.com/23zF3dF.png)

**Solução**

Sabemos que a expressão para v(t) é:

\\[
\begin{align}
    v(t) &= 26.0*exp(-20000.0*t) - 14.0*exp(-5000.0*t) V, t \geq 0 \therefore \\
    v(t) &= ({-14} e^{-5000 t} + {26} e^{-20000 t}) V, t \geq 0
\end{align}
\\]

onde \\(exp(\chi)\\) representa a função exponencial natural \\(e^{\chi}\\)

Sabemos também que 

\\[
\begin{align}
    i_{R}(t) &= \frac{v(t)}{R} = \frac{26.0*exp(-20000.0*t) - 14.0*exp(-5000.0*t)}{200} \therefore \\
    i_{R}(t) &= 0.13*exp(-20000.0*t) - 0.07*exp(-5000.0*t) \therefore \\
    i_{R}(t) &= (-70 e^{-5000t} + 130 e^{-20000 t})mA, t \geq 0
\end{align}
\\]

Agora devemos encontrar a corrente do indutor \\(I_{L}\\).

Podemos encontrá-la de duas formas:

1. Pela definição da corrente no indutor que preconiza: \\(i_{L}(t) = \frac{1}{L} \int_{0}^{t}{v(\chi) d\chi} + I_{0} \\)

2. Pela determinação da corrente no ramo capacitivo e, por fim, pela lei de kirchoff das correntes que estabelece: \\(i_{L}(t) = - i_{R}(t) - i_{C}(t)\\)

Vamos fazer a primeira forma inicialmente para validá-la com a segunda.

A primeira forma é definida nesta célula jupyter pela variável iLT:

\\[
    i_{L}(t) = -0.026*exp(-20000.0*t) + 0.056*exp(-5000.0*t)
\\]

A segunda forma, definida por:

\\[
\begin{align}
    i_{L}(t) &= - i_{R}(t) - i_{C}(t) \therefore \\
    i_{L}(t) &= - 0.13*exp(-20000.0*t) - 0.07*exp(-5000.0*t) - -0.104*exp(-20000.0*t) + 0.014*exp(-5000.0*t) \therefore \\
    i_{L}(t) &= -0.026*exp(-20000.0*t) + 0.056*exp(-5000.0*t)
\end{align}
\\]

Verifica a primeira.
